In [1]:
import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from dotenv import load_dotenv
import os


e:\project_kerja\Hackathon\GeminiGovTech\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Document


In [2]:
pdf_loader = PyPDFLoader("docs/KB GovTech.pdf")
# pdf_loader = UnstructuredPDFLoader("docs\KB GovTech.pdf")
pages = pdf_loader.load_and_split()
pages

[Document(metadata={'source': 'docs/KB GovTech.pdf', 'page': 0}, page_content='ApaituBPJS?\nBPJSadalahbadanhukumyangberoperasi sejak2014sertamemiliki wewenanguntukmemberikanjaminansosial berbentukkesehatandanketenagakerjaan. Keduaaspekinimerupakanhal pentingbagi masyarakat umumdanpekerjadalamrangkameningkatkankesejahteraannya.\nBiasanya, benefit yangakandidapatkanmasyarakat adalahasuransi kesehatansertapesangonpensiunbagi parapegawai swasta. Nah, padaartikel kali ini, OCBCakanmembahasmengenai pengertian, caradaftarbpjsonline, dancarapengecekannya. Yuk, simak!\nApaituBPJS?\nBPJSadalahsingkatandari BadanPenyelenggaraJaminanSosial, yakni lembagakhususyangbertugasuntukmenyelenggarakanjaminankesehatandanketenagakerjaanbagi masyarakat,PNS, sertapegawai swasta. Programini mulai diselenggarakanpadatahun2014melalui dasarhukumUndang-UndangNomor24Tahun2011.\nSalahsatuprogramyangdiadakanolehBPJSadalahJaminanKesehatanNasional (JKN). JKNdiselenggarakanmelalui sistemasuransi, dimanamasyarakat wajibme

## Chunking


In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500)
docs = text_splitter.split_documents(pages)

print(f"Number of documents: {len(docs)}")

Number of documents: 9


In [4]:
load_dotenv() 

#Get an API key: 
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=os.getenv("GOOGLE_API_KEY"))
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [5]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"), persist_directory="chroma")

In [6]:
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 5, "score_threshold": 0.5})

In [7]:
retrieved_docs = retriever.invoke("Apa itu bpjs?")
retrieved_docs

[Document(metadata={'page': 0, 'source': 'docs/KB GovTech.pdf'}, page_content='ApaituBPJS?\nBPJSadalahbadanhukumyangberoperasi sejak2014sertamemiliki wewenanguntukmemberikanjaminansosial berbentukkesehatandanketenagakerjaan. Keduaaspekinimerupakanhal pentingbagi masyarakat umumdanpekerjadalamrangkameningkatkankesejahteraannya.\nBiasanya, benefit yangakandidapatkanmasyarakat adalahasuransi kesehatansertapesangonpensiunbagi parapegawai swasta. Nah, padaartikel kali ini, OCBCakanmembahasmengenai pengertian, caradaftarbpjsonline, dancarapengecekannya. Yuk, simak!\nApaituBPJS?\nBPJSadalahsingkatandari BadanPenyelenggaraJaminanSosial, yakni lembagakhususyangbertugasuntukmenyelenggarakanjaminankesehatandanketenagakerjaanbagi masyarakat,PNS, sertapegawai swasta. Programini mulai diselenggarakanpadatahun2014melalui dasarhukumUndang-UndangNomor24Tahun2011.\nSalahsatuprogramyangdiadakanolehBPJSadalahJaminanKesehatanNasional (JKN). JKNdiselenggarakanmelalui sistemasuransi, dimanamasyarakat wajibme

In [8]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500, api_key=os.getenv("GOOGLE_API_KEY"))

In [139]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [9]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [9]:

system_prompt = (
    "Anda adalah asisten untuk tugas menjawab pertanyaan. "
    "Gunakan potongan konteks yang diambil berikut ini untuk menjawab "
    "pertanyaan. Jika Anda tidak tahu jawabannya, katakan bahwa Anda "
    "tidak tahu. Gunakan maksimal tiga kalimat dan buat jawaban singkat."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [10]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [143]:
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [11]:
conversational_rag_chain.invoke(
    {"input": "Apa itu BPJS?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'BPJS adalah singkatan dari Badan Penyelenggara Jaminan Sosial.  BPJS adalah lembaga yang bertugas menyelenggarakan jaminan kesehatan dan ketenagakerjaan bagi masyarakat Indonesia, PNS, dan pegawai swasta. Program ini dimulai pada tahun 2014 berdasarkan Undang-Undang Nomor 24 Tahun 2011.\n'

In [145]:
conversational_rag_chain.invoke(
    {"input": "Bagaimana cara mendaftar?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Anda dapat mendaftar BPJS melalui aplikasi JKN Mobile. Unduh dan instal aplikasi, lalu klik "Daftar", pilih "Pendaftaran Peserta Baru", dan ikuti langkah-langkah selanjutnya, termasuk memasukkan NIK, data diri, memilih faskes, dan membayar premi.\n'